In [55]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplcyberpunk

plt.style.use("cyberpunk")
import torch
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from tqdm import tqdm
import torchutils as tu
from sklearn.metrics.pairwise import cosine_similarity
torch.manual_seed(666)

In [57]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [58]:
df = pd.read_csv("/home/artemiy/recsys_project/data/dataset_itog.csv")
df.head()

,url,poster,title,description,rating,year,genres,cast,country,age
0,https://kino.mail.ru/series_915957_potapov_i_l...,https://resizer.mail.ru/p/a4bff38f-c8db-57ac-9...,Гадалка,"Люся видит во сне мертвое тело своей сестры, б...",6.4,2019,"Драма, Детектив, Мистика","Михаил Пореченков, Екатерина Олькина, Яна Край...",Россия,16 +
1,https://kino.mail.ru/series_773009_ohotnik/,https://resizer.mail.ru/p/dccc9300-db25-55ad-b...,Охотник,"Средневековый охотник — Владимир, сын Казатула...",7.4,2006,"Фантастика, Фэнтези, Приключения","Евгений Цыганов, Елена Панова, Сергей Никоненк...",Россия,16 +
2,https://kino.mail.ru/series_925738_velikaya/,https://resizer.mail.ru/p/2bebfd49-f476-5000-9...,Великая,Сериал о молодости императрицы Екатерины II. М...,4.8,2020,"Драма, Комедия","Эль Фаннинг, Николас Холт, Фиби Фокс, Адам Год...",США,18 +
3,https://kino.mail.ru/series_784898_fantastiche...,https://resizer.mail.ru/p/68617b86-fd8b-5e43-b...,Фантастическая девушка,Главная героиня сериала — Эви — фантастическая...,6.5,1987,"Фантастика, Комедия, Семейный","Донна Пескоу, Морин Флэннигэн, Берт Рейнолдс, ...",США,0 +
4,https://kino.mail.ru/series_785356_shpionka/,https://resizer.mail.ru/p/733fedb7-047d-58d5-9...,Шпионка,"Переезды по всему миру, борьба с «плохими парн...",7.1,2001,"Драма, Боевик, Криминал, Триллер, Детектив","Дженнифер Гарнер, Майкл Вартан, Рон Рифкин, Ви...",США,16 +


In [59]:
# pip install transformers sentencepiece
# import torch
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")
model.cuda()  # uncomment it if you have a GPU


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(83828, 312, padding_idx=0)
    (position_embeddings): Embedding(2048, 312)
    (token_type_embeddings): Embedding(2, 312)
    (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-2): 3 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=312, out_features=312, bias=True)
            (key): Linear(in_features=312, out_features=312, bias=True)
            (value): Linear(in_features=312, out_features=312, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=312, out_features=312, bias=True)
            (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)

In [60]:
def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()


print(embed_bert_cls("привет мир", model, tokenizer).shape)

(312,)


In [61]:
df.columns

Index(['url', 'poster', 'title', 'description', 'rating', 'year', 'genres',
       'cast', 'country', 'age'],
      dtype='object')

In [71]:
df['description'].isna().sum()

0

In [63]:
df["description"] = df["description"].fillna("")
df["description"] = df["description"].astype(str)

In [64]:
# Создание эмбеддингов для описаний сериалов
df["description_embedding"] = df["description"].apply(
    lambda x: embed_bert_cls(x, model, tokenizer)
)

In [65]:
def find_shows(query, df, model, tokenizer, top_k=5):

    # Создание эмбеддинга для пользовательского запроса
    query_embedding = embed_bert_cls(query, model, tokenizer).reshape(1, -1)

    # Получение эмбеддингов описаний сериалов
    embeddings = np.stack(df["description_embedding"].values)

    # Вычисление косинусного сходства
    similarities = cosine_similarity(query_embedding, embeddings).flatten()

    # Получение топ-K самых похожих сериалов
    top_k_indices = similarities.argsort()[-top_k:][::-1]

    # Возвращаем отфильтрованный и отсортированный DataFrame
    result = df.iloc[top_k_indices].copy()
    result["similarity"] = similarities[top_k_indices]

    # Удаление столбца description_embedding
    result = result.drop(columns=["description_embedding"])

    return result.sort_values("similarity", ascending=False)

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15750 entries, 0 to 15749
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   url                    15750 non-null  object 
 1   poster                 15750 non-null  object 
 2   title                  15750 non-null  object 
 3   description            15750 non-null  object 
 4   rating                 5737 non-null   float64
 5   year                   15750 non-null  int64  
 6   genres                 15744 non-null  object 
 7   cast                   15653 non-null  object 
 8   country                15647 non-null  object 
 9   age                    15617 non-null  object 
 10  description_embedding  15750 non-null  object 
dtypes: float64(1), int64(1), object(9)
memory usage: 1.3+ MB


In [70]:
query = 'ужасы'
find__shows(query, df, model, tokenizer, 10)

,url,poster,title,description,rating,year,genres,cast,country,age,description_embedding
7874,https://kino.mail.ru/series_915186_pulittserov...,/img/v2/nopicture/308x462.png,Пулитцеровский театр,,NaN,1950,Драма,NaN,NaN,NaN,"[-0.018243225, -0.018014258, -0.014497836, -0...."
7803,https://kino.mail.ru/series_911009_o_hara_kazn...,/img/v2/nopicture/308x462.png,"О'Хара, Казначейство США",,NaN,1971,Драма,NaN,NaN,NaN,"[-0.018243225, -0.018014258, -0.014497836, -0...."
7788,https://kino.mail.ru/series_911044_osvobodite_...,/img/v2/nopicture/308x462.png,Освободите место для дедули,,NaN,1970,Комедия,NaN,NaN,NaN,"[-0.018243225, -0.018014258, -0.014497836, -0...."
7787,https://kino.mail.ru/series_914867_osvobodite_...,/img/v2/nopicture/308x462.png,Освободите место для папочки,,NaN,1953,"Комедия, Семейный",NaN,NaN,NaN,"[-0.018243225, -0.018014258, -0.014497836, -0...."
7785,https://kino.mail.ru/series_915135_opoznanie/,/img/v2/nopicture/308x462.png,Опознание,,NaN,1954,"Драма, Криминал",NaN,NaN,NaN,"[-0.018243225, -0.018014258, -0.014497836, -0...."
7770,https://kino.mail.ru/series_915136_oblava/,/img/v2/nopicture/308x462.png,Облава,,NaN,1951,"Драма, Криминал, Детектив",NaN,NaN,NaN,"[-0.018243225, -0.018014258, -0.014497836, -0...."
7765,https://kino.mail.ru/series_911041_nenni_i_pro...,/img/v2/nopicture/308x462.png,Нэнни и профессор,,NaN,1970,"Комедия, Семейный",NaN,NaN,NaN,"[-0.018243225, -0.018014258, -0.014497836, -0...."
7758,https://kino.mail.ru/series_911017_novoe_shou_...,/img/v2/nopicture/308x462.png,Новое шоу Дика Ван Дайка,,NaN,1971,Комедия,NaN,NaN,NaN,"[-0.018243225, -0.018014258, -0.014497836, -0...."
7732,https://kino.mail.ru/series_915149_nasha_miss_...,/img/v2/nopicture/308x462.png,Наша мисс Брукс,,NaN,1952,"Комедия, Семейный",NaN,NaN,NaN,"[-0.018243225, -0.018014258, -0.014497836, -0...."
7725,https://kino.mail.ru/series_911174_naimenovani...,/img/v2/nopicture/308x462.png,Наименование игры,,NaN,1968,Приключения,NaN,NaN,NaN,"[-0.018243225, -0.018014258, -0.014497836, -0...."
